In [ ]:
import gradio as gr

def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)

demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)

demo.launch()


In [ ]:
gr.themes.builder()


In [ ]:
import gradio as gr
from gradio.themes.base import Base
import time

class Seafoam(Base):
    pass

seafoam = Seafoam()

with gr.Blocks(theme=seafoam) as demo:
    textbox = gr.Textbox(label="Name")
    slider = gr.Slider(label="Count", minimum=0, maximum=100, step=1)
    with gr.Row():
        button = gr.Button("Submit", variant="primary")
        clear = gr.Button("Clear")
    output = gr.Textbox(label="Output")

    def repeat(name, count):
        time.sleep(3)
        return name * count
    
    button.click(repeat, [textbox, slider], output)

demo.launch()


In [ ]:
import gradio as gr

with gr.Blocks(theme=gr.themes.Soft()) as demo:
     def greet(name, intensity):
    return "Hello, " + name + "!" * int(intensity)


demo = gr.Interface(
    fn=greet,
    inputs=["text", "slider"],
    outputs=["text"],
)


demo.launch()

In [ ]:
import gradio as gr

title = "Multiple Interfaces"

#app 1
def user_greeting(name):
    return "Hi! " + name + " Welcome to your first Gradio application!😎"

#app 2
def user_help(do):
    return "So today we will do " + do + "using Gradio. Great choice!"

#interface 1
app1 =  gr.Interface(fn = user_greeting, inputs="text", outputs="text")
#interface 2

app2 =  gr.Interface(fn = user_help, inputs="text", outputs="text")

demo = gr.TabbedInterface([app1, app2], ["Welcome", "What to do"])

demo.launch()

In [ ]:
import gradio as gr
import numpy as np

In [ ]:
def reverse_audio(audio):
    sr, data = audio
    return (sr, np.flipud(data))

In [ ]:
input_audio = gr.Audio(sources=["microphone"])
demo = gr.Interface(
    fn=reverse_audio,
    inputs=input_audio,
    outputs="audio"
)


In [ ]:
if __name__ == "__main__":
    demo.launch()

Speech to Audio with Gradio ASR and transformers

In [1]:
import gradio as gr
from transformers import pipeline
import numpy as np

# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio # Extract sampling rate and audio data
    y = y.astype(np.float32) # Convert audio data to float32
    y /= np.max(np.abs(y)) # Normalize the audio data
    return transcriber({"sampling_rate": sr, "raw": y})["text"] # Transcribe the audio


C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
demo = gr.Interface(
    transcribe,
    gr.Audio(sources=["microphone"]),
    "text",
)


In [4]:
demo.launch()

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 407, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await sel

Gradio Interface with ASR returning text to dataframe

In [36]:
import gradio as gr
from transformers import pipeline
import numpy as np
import pandas as pd

# Initialize the ASR pipeline
transcriber = pipeline("automatic-speech-recognition", model="openai/whisper-base.en")

def transcribe(audio):
    sr, y = audio
    y = y.astype(np.float32)
    y /= np.max(np.abs(y))
    return transcriber({"sampling_rate": sr, "raw": y})["text"]


In [22]:
#create dataframe from transcription

def transcribe_to_df(audio):
    transcribed_text = transcribe(audio)
    # Create a DataFrame with the transcribed text
    df = pd.DataFrame({"Transcription": [transcribed_text]})
    return df


In [37]:
demo = gr.Interface(
    transcribe_to_df,
    gr.Audio(sources=["microphone"]),
    "dataframe",
)

In [38]:
demo.launch()


Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\uvicorn\protocols\http\h11_impl.py", line 407, in run_asgi
    result = await app(  # type: ignore[func-returns-value]
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\uvicorn\middleware\proxy_headers.py", line 69, in __call__
    return await self.app(scope, receive, send)
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\fastapi\applications.py", line 1054, in __call__
    await super().__call__(scope, receive, send)
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\starlette\applications.py", line 123, in __call__
    await self.middleware_stack(scope, receive, send)
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 186, in __call__
    raise exc
  File "C:\Users\c_mar\anaconda3\envs\dev\lib\site-packages\starlette\middleware\errors.py", line 164, in __call__
    await sel